<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Tradevesting/myProspects_Upside_Analysis_ATH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pytz
tz_NY = pytz.timezone('Asia/Kolkata')
datetime_NY = datetime.datetime.now(tz_NY)
print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

Run date time (IST): 2023-06-29 11:05:52


In [2]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218908 sha256=9287c038ac64c4c7ef42350a34ae8112078e19a0d9eb33bd69c39119118f061a
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas-ta


In [3]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-365*5)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [4]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

mypf = pd.read_csv('/content/drive/My Drive/data/stocks/myProspectsScrips.csv')

#mypf = mypf[mypf['Nifty100'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

stock_ids = stock_n100

#mypf = mypf[mypf.Forecast.notnull()]
#stock_ids = mypf['Symbol'].unique()

stock_ids.sort()

Mounted at /content/drive


In [5]:
# star stocks & my stocks
star_stocks = list(mypf[mypf['StarStock'] == 1]['Stock'].values)
my_stocks = list(mypf[mypf['InPortfolio'] == 1]['Symbol'].values)

In [6]:
# conviction distribution
mypf['Conviction'].value_counts()

M    80
L    65
H    38
Name: Conviction, dtype: int64

In [7]:
# star stocks
mypf['StarStock'].value_counts()

0.0    93
1.0    83
Name: StarStock, dtype: int64

In [8]:
# v40/v40n stocks
mypf['Criteria'].value_counts()

V40N    39
V40     38
2T3Y    10
SS       7
V200     6
TBD      2
Name: Criteria, dtype: int64

In [9]:
# latest quarter
mypf['LatestQtr'].value_counts()

1.0    165
0.0     16
Name: LatestQtr, dtype: int64

In [10]:
# star + v40/v40n stocks
mypf[(mypf['Criteria'] == 'V40') | (mypf['Criteria'] == 'V40N')]['StarStock'].value_counts()

1.0    41
0.0    36
Name: StarStock, dtype: int64

In [11]:
# stocks in portfolio
mypf['InPortfolio'].value_counts()

0    139
1     44
Name: InPortfolio, dtype: int64

In [12]:
# star stocks in portfolio
mypf[(mypf['InPortfolio'] == 1)]['StarStock'].value_counts()

1.0    27
0.0    13
Name: StarStock, dtype: int64

In [13]:
# criteria of portfolio stocks
mypf[(mypf['InPortfolio'] == 1)]['Criteria'].value_counts()

V40     12
V40N    10
2T3Y    10
SS       6
V200     3
TBD      2
Name: Criteria, dtype: int64

In [14]:
# strategies in portfolio stocks
mypf[(mypf['InPortfolio'] == 1)]['Strategy'].value_counts()

ATH     28
2T3Y    14
Name: Strategy, dtype: int64

In [15]:
# import libraries

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib
yfin.pdr_override()
import pandas_ta as ta

In [16]:
# fetch current PE
import requests
from bs4 import BeautifulSoup

def get_current_pe(stock_name):

  link_blueprint = "https://www.screener.in/company/"

  stocks = ['INDIGOPNTS', 'IRCTC', 'GILLETTE', 'PAGEIND']

  if stock_name in stocks:
    actual_link = link_blueprint + stock_name.upper()
  else:
    actual_link = link_blueprint + stock_name.upper() + "/consolidated/"

  page = requests.get(actual_link)
  #print(actual_link)
  link_soup = BeautifulSoup(page.content, 'html.parser')

  test_list = link_soup.find(id="top").text.replace("  ", "").split('\n')

  while("" in test_list) :
      test_list.remove("")
  while("\r" in test_list) :
      test_list.remove("\r")
  while(" \r" in test_list) :
      test_list.remove(" \r")

  try:
    current_pe = float(test_list[test_list.index('Stock P/E')+1:test_list.index('Stock P/E')+2][0])
  except ValueError as ve:
    current_pe = 1000

  return current_pe


In [17]:
# function to compute stock attributes

def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))
    stock_df = yfin.Ticker(stock_symbol).history(period='5y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    #stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe
    #stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    # column names for long and short moving average columns
    short_window_col = str(short_window) + '_' + moving_avg
    mid_window_col = str(mid_window) + '_' + moving_avg
    long_window_col = str(long_window) + '_' + moving_avg

    #print(stock_df)

    # Create short exponential moving average column
    stock_df[short_window_col] = round(stock_df['Close'].rolling(window = short_window, min_periods = 1).mean(),0)

    stock_df[mid_window_col] = round(stock_df['Close'].rolling(window = mid_window, min_periods = 1).mean(),0)

    #stock_df['RSI'] = round(talib.RSI(stock_df['Close Price'], timeperiod=14),0)

    # Create a long exponential moving average column
    stock_df[long_window_col] = round(stock_df['Close'].rolling(window = long_window, min_periods = 1).mean(),0)
    stock_df['Stock'] = stock_symbol
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df['200_SMA_Lag10'] = stock_df['200_SMA'].shift(10)
    stock_df['Slope%_200'] = round((stock_df['200_SMA'] - stock_df['200_SMA_Lag10'])*100/stock_df['200_SMA'],2)
    stock_df['50_SMA_Lag5'] = stock_df['50_SMA'].shift(5)
    stock_df['Slope%_50'] = round((stock_df['50_SMA'] - stock_df['50_SMA_Lag5'])*100/stock_df['50_SMA'],2)

    #stock_df.ta.rsi(append=True)
    #stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
    stock_df['Close'] = round(stock_df['Close'],0)
    #stock_df['RSI'] = round(stock_df['RSI'],0)
    stock_df['Max'] = round(max(stock_df['Close']),0)
    stock_df['Min'] = round(min(stock_df['Close']),0)
    stock_df['Discount%'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df['Upside%ATH'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Close']),0)
    stock_df.drop(['Open', 'Low', 'High', 'Volume','20_SMA','50_SMA','200_SMA','200_SMA_Lag10','50_SMA_Lag5'
                   ], axis=1, inplace=True)

    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:
    #print(stock_id)
    if stock_id in my_stocks:
      current_pe = get_current_pe(stock_id)
    else:
      current_pe = 1000

    stock_id = stock_id.upper() + '.NS'
    tmp = stock_prec_dev(stock_id)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Avg_Cost'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['AvgCost'].values[0]
    tmp['Shares'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Shares'].values[0]
    tmp['Med_PE'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['MedianPE'].values[0]
    tmp['Curr_PE'] = current_pe
    #tmp['BizOutlook'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['BizOutlook'].values[0]
    #tmp['BizDurability'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['BizDurability'].values[0]
    tmp['Conviction'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Conviction'].values[0]
    tmp['LatestQtr'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['LatestQtr'].values[0]
    tmp['StarStock'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['StarStock'].values[0]
    tmp['Criteria'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Criteria'].values[0]
    tmp['Strategy'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Strategy'].values[0]
    df_prec_dev = df_prec_dev.append(tmp)

In [18]:
# nifty in last 5 weeks
stock_prec_dev('^NSEI')[-21::5]

,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH
Date,,,,,,,,,
2023-05-31 00:00:00+05:30,18534.0,^NSEI,3.89,0.37,0.72,18972.0,7610.0,3.85,2.0
2023-06-07 00:00:00+05:30,18726.0,^NSEI,4.83,0.29,0.85,18972.0,7610.0,2.17,1.0
2023-06-14 00:00:00+05:30,18756.0,^NSEI,4.86,0.26,0.90,18972.0,7610.0,1.90,1.0
2023-06-21 00:00:00+05:30,18857.0,^NSEI,5.24,0.30,0.70,18972.0,7610.0,1.01,1.0
2023-06-28 00:00:00+05:30,18972.0,^NSEI,5.71,0.33,0.57,18972.0,7610.0,0.00,0.0


In [19]:
stock_ids = df_prec_dev['Stock'].values
df_prec_dev.reset_index(drop=True, inplace=True)
df_prec_dev.drop(['Date'], axis=1, inplace=True)

mypf['Stock'] = mypf['Symbol'] + '.NS'
#df_prec_dev.drop(['Med_PE'], axis=1, inplace=True)

In [20]:
df_prec_dev['Discount%PE'] = round((df_prec_dev['Med_PE']-df_prec_dev['Curr_PE'])*100/(df_prec_dev['Med_PE']),0)
#df_prec_dev['Upside%Estimate'] = round((df_prec_dev['Max']-df_prec_dev['Avg_Cost'])*100/(df_prec_dev['Avg_Cost']),0)
df_prec_dev = df_prec_dev.set_index('Stock')

In [21]:
# portfolio stocks
cols = ['Close',	'Dev%_200',	'Slope%_200',	'Slope%_50', 'Discount%',	'Upside%ATH',	'Avg_Cost',	'Med_PE',	'Curr_PE',	'Conviction', 'Discount%PE', 'LatestQtr', 'StarStock', 'Criteria', 'Strategy']
tmp_df = df_prec_dev[cols]
tmp_df = tmp_df[tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 44


,Close,Dev%_200,Slope%_200,Slope%_50,Discount%,Upside%ATH,Avg_Cost,Med_PE,Curr_PE,Conviction,Discount%PE,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
GLAND.NS,1060.0,-29.89,-4.70,-2.11,95.15,307.0,1225.87,45.2,21.30,M,53.0,1.0,0.0,2T3Y,2T3Y
VALIANTORG.NS,557.0,-1.94,-0.53,1.31,87.97,232.0,542.52,24.0,17.90,M,25.0,0.0,0.0,2T3Y,2T3Y
VAIBHAVGBL.NS,309.0,-3.36,-0.31,0.00,76.48,213.0,317.48,35.4,48.50,M,-37.0,1.0,0.0,2T3Y,2T3Y
BANDHANBNK.NS,237.0,-1.54,-0.83,0.82,85.41,205.0,253.98,NaN,1000.00,L,NaN,1.0,1.0,2T3Y,2T3Y
LUXIND.NS,1498.0,-2.54,-0.91,2.05,82.45,203.0,1493.72,21.4,32.80,M,-53.0,1.0,0.0,V40N,2T3Y
NIACL.NS,117.0,9.53,1.87,0.86,83.26,166.0,117.07,NaN,18.40,M,NaN,1.0,0.0,TBD,ATH
METROPOLIS.NS,1459.0,6.40,0.07,1.22,77.87,136.0,1365.98,NaN,52.30,M,NaN,NaN,NaN,NaN,ATH
INDIGOPNTS.NS,1425.0,9.87,-0.77,2.60,79.28,118.0,1407.73,104.9,51.40,M,51.0,1.0,1.0,V40N,2T3Y
TEAMLEASE.NS,2471.0,-1.75,-1.95,1.07,73.93,114.0,2279.93,NaN,37.00,M,NaN,1.0,1.0,V40N,2T3Y


In [22]:
# top v40/v40n stocks from portfolio for SIP
tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')][tmp_df['Dev%_200'] < 0].sort_values(by = 'Upside%ATH', ascending=False)

,Close,Dev%_200,Slope%_200,Slope%_50,Discount%,Upside%ATH,Avg_Cost,Med_PE,Curr_PE,Conviction,Discount%PE,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
LUXIND.NS,1498.0,-2.54,-0.91,2.05,82.45,203.0,1493.72,21.4,32.8,M,-53.0,1.0,0.0,V40N,2T3Y
TEAMLEASE.NS,2471.0,-1.75,-1.95,1.07,73.93,114.0,2279.93,NaN,37.0,M,NaN,1.0,1.0,V40N,2T3Y
RAJESHEXPO.NS,542.0,-18.98,-0.15,-0.70,79.96,75.0,624.78,NaN,11.2,M,NaN,1.0,1.0,V40N,ATH
SUNTV.NS,435.0,-6.58,-0.64,0.23,59.65,63.0,446.75,NaN,10.0,M,NaN,1.0,1.0,V40N,ATH
FINEORG.NS,4929.0,-8.28,-1.23,0.87,35.09,46.0,4697.43,NaN,24.4,M,NaN,1.0,1.0,V40N,ATH
INFY.NS,1296.0,-7.95,-0.50,-0.24,40.35,43.0,1263.00,29.0,22.4,H,23.0,1.0,1.0,V40,ATH
PAGEIND.NS,37499.0,-12.25,-1.36,0.02,42.85,43.0,39325.03,97.7,73.1,H,25.0,1.0,1.0,V40,ATH
BAYERCROP.NS,4359.0,-2.04,-1.03,0.48,50.77,38.0,4185.24,NaN,397.0,M,NaN,1.0,1.0,V40N,ATH


In [23]:
# top 10 from portfolio for SIP
tmp_df[tmp_df['Dev%_200'] < 0].sort_values(by = 'Upside%ATH', ascending=False).head(10)

,Close,Dev%_200,Slope%_200,Slope%_50,Discount%,Upside%ATH,Avg_Cost,Med_PE,Curr_PE,Conviction,Discount%PE,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
GLAND.NS,1060.0,-29.89,-4.70,-2.11,95.15,307.0,1225.87,45.2,21.3,M,53.0,1.0,0.0,2T3Y,2T3Y
VALIANTORG.NS,557.0,-1.94,-0.53,1.31,87.97,232.0,542.52,24.0,17.9,M,25.0,0.0,0.0,2T3Y,2T3Y
VAIBHAVGBL.NS,309.0,-3.36,-0.31,0.00,76.48,213.0,317.48,35.4,48.5,M,-37.0,1.0,0.0,2T3Y,2T3Y
BANDHANBNK.NS,237.0,-1.54,-0.83,0.82,85.41,205.0,253.98,NaN,1000.0,L,NaN,1.0,1.0,2T3Y,2T3Y
LUXIND.NS,1498.0,-2.54,-0.91,2.05,82.45,203.0,1493.72,21.4,32.8,M,-53.0,1.0,0.0,V40N,2T3Y
TEAMLEASE.NS,2471.0,-1.75,-1.95,1.07,73.93,114.0,2279.93,NaN,37.0,M,NaN,1.0,1.0,V40N,2T3Y
AAVAS.NS,1540.0,-14.52,-2.33,-2.10,64.66,108.0,1390.60,NaN,28.2,M,NaN,1.0,1.0,2T3Y,2T3Y
CLEAN.NS,1392.0,-6.95,-1.34,-0.42,89.14,90.0,1396.49,NaN,50.1,L,NaN,0.0,NaN,2T3Y,2T3Y
WIPRO.NS,382.0,-2.23,-0.26,0.52,59.86,87.0,388.07,20.9,18.5,L,11.0,1.0,0.0,2T3Y,2T3Y


In [24]:
# top 5 from portfolio to book profit
tmp_df.sort_values(by = 'Upside%ATH', ascending=True).head(7)

,Close,Dev%_200,Slope%_200,Slope%_50,Discount%,Upside%ATH,Avg_Cost,Med_PE,Curr_PE,Conviction,Discount%PE,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
ASIANPAINT.NS,3350.0,10.56,-0.10,1.64,7.86,6.0,3091.34,87.9,77.7,H,12.0,1.0,1.0,V40,ATH
RELIANCE.NS,2530.0,2.78,-0.12,0.65,14.45,11.0,2467.15,29.1,25.7,H,12.0,1.0,1.0,V40,ATH
BAJFINANCE.NS,7070.0,7.80,0.00,1.68,13.16,11.0,6602.15,52.6,37.3,H,29.0,1.0,1.0,V40,ATH
PIDILITIND.NS,2603.0,2.48,-0.20,1.02,14.92,11.0,2504.06,94.6,104.0,H,-10.0,1.0,1.0,V40,ATH
TCS.NS,3221.0,0.44,0.22,0.19,27.44,20.0,3231.00,28.1,27.9,H,1.0,1.0,1.0,V40,ATH
ICICIGI.NS,1330.0,15.22,0.17,1.82,30.30,21.0,1137.80,46.2,1000.0,M,-2065.0,1.0,1.0,V40,ATH
BAJAJFINSV.NS,1520.0,1.59,-0.53,1.26,25.78,25.0,1454.93,39.2,37.7,M,4.0,1.0,1.0,V40,ATH


In [25]:
df_prec_dev.drop([ 'Avg_Cost', 'Shares', 'Med_PE', 'Curr_PE', 'Discount%PE'], axis=1, inplace=True)

In [26]:
# star + v40/v40n stocks (for upside)

tmp_df = df_prec_dev[df_prec_dev.index.isin(star_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 26)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 2


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,
DIXON.NS,4375.0,19.21,0.49,4.01,5750.0,374.0,25.58,31.0,M,1.0,1.0,V40N,NaN
BERGEPAINT.NS,674.0,11.38,0.17,1.57,858.0,260.0,30.77,27.0,M,1.0,1.0,V40,NaN


In [27]:
# top 5 from star + v40/v40n for SIP
tmp_df.sort_values(by = 'Dev%_200', ascending=True).head(5)

,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,
BERGEPAINT.NS,674.0,11.38,0.17,1.57,858.0,260.0,30.77,27.0,M,1.0,1.0,V40,NaN
DIXON.NS,4375.0,19.21,0.49,4.01,5750.0,374.0,25.58,31.0,M,1.0,1.0,V40N,NaN


In [28]:
# star + v40/v40n stocks (for breakout)

tmp_df = df_prec_dev[df_prec_dev.index.isin(star_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[(tmp_df['Upside%ATH'] < 5)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 12


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,
AKZOINDIA.NS,2424.0,7.36,1.15,0.62,2508.0,1360.0,7.32,3.0,M,1.0,1.0,V40,NaN
CERA.NS,7781.0,28.95,2.29,1.87,8036.0,2065.0,4.27,3.0,H,1.0,1.0,V40N,NaN
BAJAJ-AUTO.NS,4582.0,19.94,0.79,0.86,4655.0,1748.0,2.51,2.0,L,1.0,1.0,V40,NaN
COLPAL.NS,1690.0,9.60,0.26,0.87,1719.0,931.0,3.68,2.0,H,1.0,1.0,V40,NaN
HDFCBANK.NS,1673.0,6.16,0.51,0.00,1708.0,747.0,3.64,2.0,H,1.0,1.0,V40,NaN
EQUITASBNK.NS,88.0,40.16,3.17,2.47,89.0,33.0,1.79,1.0,M,1.0,1.0,V40N,NaN
HINDUNILVR.NS,2670.0,4.36,0.23,0.50,2706.0,1394.0,2.74,1.0,H,1.0,1.0,V40,NaN
ICICIBANK.NS,940.0,4.91,0.33,0.43,954.0,252.0,1.99,1.0,H,1.0,1.0,V40,NaN
ITC.NS,449.0,22.01,1.90,1.39,454.0,124.0,1.52,1.0,H,1.0,1.0,V40,NaN


In [29]:
# star stocks (for upside)

tmp_df = df_prec_dev[df_prec_dev.index.isin(star_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['Criteria'] != 'V40') & (tmp_df['Criteria'] != 'V40N')]
#tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] != -1)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 25) & (tmp_df['Discount%'] >= 60)]
tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 20)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 8


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,
ADANITRANS.NS,822.0,-58.55,-7.67,-2.87,4105.0,148.0,82.97,399.0,L,1.0,1.0,NaN,NaN
ADANIGREEN.NS,960.0,-34.16,-4.80,0.21,2970.0,25.0,68.25,209.0,L,1.0,1.0,NaN,NaN
ADANIENT.NS,2413.0,-14.37,-1.56,2.14,4165.0,66.0,42.74,73.0,M,1.0,1.0,NaN,NaN
SBICARD.NS,852.0,4.51,-0.25,1.16,1133.0,496.0,44.11,33.0,M,1.0,1.0,NaN,NaN
TATAPOWER.NS,219.0,3.63,-0.47,1.43,285.0,26.0,25.48,30.0,M,1.0,1.0,NaN,NaN
GODREJIND.NS,502.0,13.22,0.23,1.06,644.0,235.0,34.72,28.0,L,1.0,1.0,NaN,NaN
INDIANB.NS,278.0,5.86,1.90,-0.35,346.0,39.0,22.15,24.0,H,1.0,1.0,NaN,NaN
UPL.NS,682.0,-5.31,-0.56,-0.86,832.0,245.0,25.55,22.0,L,1.0,1.0,NaN,NaN


In [30]:
# star stocks (for breakout)

tmp_df = df_prec_dev[df_prec_dev.index.isin(star_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
#tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] != -1)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 25) & (tmp_df['Discount%'] >= 60)]
tmp_df = tmp_df[(tmp_df['Upside%ATH'] < 5)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=True)

qualified stocks: 12


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,
ABBOTINDIA.NS,23278.0,12.19,1.02,0.10,23278.0,6634.0,0.00,0.0,M,1.0,1.0,V40,NaN
POLYCAB.NS,3631.0,24.85,1.82,1.30,3631.0,523.0,0.00,0.0,M,1.0,1.0,V40N,NaN
TITAN.NS,3031.0,16.67,0.73,1.44,3031.0,730.0,0.00,0.0,H,1.0,1.0,V40,NaN
EQUITASBNK.NS,88.0,40.16,3.17,2.47,89.0,33.0,1.79,1.0,M,1.0,1.0,V40N,NaN
HINDUNILVR.NS,2670.0,4.36,0.23,0.50,2706.0,1394.0,2.74,1.0,H,1.0,1.0,V40,NaN
ICICIBANK.NS,940.0,4.91,0.33,0.43,954.0,252.0,1.99,1.0,H,1.0,1.0,V40,NaN
ITC.NS,449.0,22.01,1.90,1.39,454.0,124.0,1.52,1.0,H,1.0,1.0,V40,NaN
BAJAJ-AUTO.NS,4582.0,19.94,0.79,0.86,4655.0,1748.0,2.51,2.0,L,1.0,1.0,V40,NaN
COLPAL.NS,1690.0,9.60,0.26,0.87,1719.0,931.0,3.68,2.0,H,1.0,1.0,V40,NaN


In [31]:
# stocks excluded from all time high

excluded_stocks = ['ACC.NS','AMBUJACEM.NS','CADILAHC.NS','GILLETTE.NS','NESTLEIND.NS','PGHH.NS', 'PVRINOX.NS', 'SIEMENS.NS']

tmp_df = df_prec_dev[df_prec_dev.index.isin(excluded_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
#tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] != -1)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 25) & (tmp_df['Discount%'] >= 60)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 20)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 2


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,
PGHH.NS,14472.0,4.22,0.07,0.14,15778.0,8145.0,17.11,9.0,L,0.0,0.0,V40,NaN
NESTLEIND.NS,22750.0,14.04,0.87,1.29,22969.0,8621.0,1.53,1.0,H,0.0,0.0,V40,NaN


In [32]:
# stocks prospects

excluded_stocks = ['BANDHANBNK.NS','GICRE.NS','JMFINANCIL.NS','NIACL.NS','PNB.NS','UJJIVANSFB.NS']

tmp_df = df_prec_dev[df_prec_dev.index.isin(excluded_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
#tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
#tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] != -1)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 25) & (tmp_df['Discount%'] >= 60)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 20)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 0


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,


In [33]:
# prospects with heavy upside
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
#tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] == 1)]
tmp_df = tmp_df[tmp_df['Upside%ATH'] > 40]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 11


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,
JCHAC.NS,1038.0,-11.32,-2.05,-0.65,3219.0,1017.0,99.05,210.0,M,1.0,0.0,V40N,NaN
LALPATHLAB.NS,2204.0,1.61,-0.83,1.39,4186.0,807.0,58.66,90.0,M,1.0,0.0,V40N,NaN
WHIRLPOOL.NS,1464.0,0.84,-1.24,0.93,2667.0,1254.0,85.14,82.0,M,1.0,0.0,V40,NaN
NAM-INDIA.NS,251.0,4.38,-0.83,0.85,425.0,115.0,56.13,69.0,L,1.0,0.0,V40,NaN
SYMPHONY.NS,899.0,-4.56,0.00,-0.99,1470.0,707.0,74.84,64.0,M,1.0,0.0,V40N,NaN
SFL.NS,1210.0,-1.07,-1.39,1.02,1972.0,566.0,54.20,63.0,M,1.0,0.0,V40N,NaN
TTKPRESTIG.NS,725.0,-10.27,-1.36,0.28,1165.0,391.0,56.85,61.0,M,1.0,0.0,V40N,NaN
PFIZER.NS,3845.0,-5.18,-0.47,0.18,6009.0,2283.0,58.08,56.0,L,1.0,0.0,V40,NaN
MOTILALOFS.NS,705.0,5.89,-0.45,0.78,1095.0,441.0,59.63,55.0,L,1.0,0.0,V40N,NaN


In [34]:
# prospects near 52W low
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
#tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] == 1)]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[tmp_df['Discount%'] > 60]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Discount%', ascending=False)

qualified stocks: 4


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,
JCHAC.NS,1038.0,-11.32,-2.05,-0.65,3219.0,1017.0,99.05,210.0,M,1.0,0.0,V40N,NaN
WHIRLPOOL.NS,1464.0,0.84,-1.24,0.93,2667.0,1254.0,85.14,82.0,M,1.0,0.0,V40,NaN
SYMPHONY.NS,899.0,-4.56,0.00,-0.99,1470.0,707.0,74.84,64.0,M,1.0,0.0,V40N,NaN
SIS.NS,410.0,5.40,-0.51,1.01,605.0,321.0,68.66,48.0,M,1.0,0.0,V40N,NaN


In [35]:
# prospects near 52W high
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
#tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] == 1)]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[tmp_df['Discount%'] < 15]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Discount%', ascending=True)

qualified stocks: 22


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,
TITAN.NS,3031.0,16.67,0.73,1.44,3031.0,730.0,0.00,0.0,H,1.0,1.0,V40,NaN
ABBOTINDIA.NS,23278.0,12.19,1.02,0.10,23278.0,6634.0,0.00,0.0,M,1.0,1.0,V40,NaN
POLYCAB.NS,3631.0,24.85,1.82,1.30,3631.0,523.0,0.00,0.0,M,1.0,1.0,V40N,NaN
AXISBANK.NS,980.0,11.54,1.25,1.19,986.0,303.0,0.88,1.0,M,1.0,0.0,V40,NaN
ITC.NS,449.0,22.01,1.90,1.39,454.0,124.0,1.52,1.0,H,1.0,1.0,V40,NaN
NESTLEIND.NS,22750.0,14.04,0.87,1.29,22969.0,8621.0,1.53,1.0,H,0.0,0.0,V40,NaN
EQUITASBNK.NS,88.0,40.16,3.17,2.47,89.0,33.0,1.79,1.0,M,1.0,1.0,V40N,NaN
ICICIBANK.NS,940.0,4.91,0.33,0.43,954.0,252.0,1.99,1.0,H,1.0,1.0,V40,NaN
BAJAJ-AUTO.NS,4582.0,19.94,0.79,0.86,4655.0,1748.0,2.51,2.0,L,1.0,1.0,V40,NaN


In [36]:
# prospects in bull run
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
#tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] == 1)]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[(tmp_df['Slope%_200'] > 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_200'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 23


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,
DIXON.NS,4375.0,19.21,0.49,4.01,5750.0,374.0,25.58,31.0,M,1.0,1.0,V40N,NaN
ANGELONE.NS,1703.0,27.18,1.12,2.93,1889.0,207.0,11.06,11.0,M,1.0,1.0,V40N,NaN
ISEC.NS,616.0,23.93,0.60,2.61,829.0,164.0,32.03,35.0,M,1.0,0.0,V40N,NaN
CAPLIPOINT.NS,803.0,13.11,0.28,2.31,947.0,190.0,19.02,18.0,M,1.0,1.0,V40N,NaN
HDFCLIFE.NS,660.0,20.38,0.73,2.09,754.0,339.0,22.65,14.0,H,1.0,0.0,V40,NaN
OFSS.NS,3848.0,25.48,1.43,2.05,4392.0,1232.0,17.22,14.0,M,1.0,1.0,V40N,NaN
SANOFI.NS,6910.0,22.84,0.96,2.00,8070.0,3825.0,27.33,17.0,M,0.0,0.0,V40,NaN
MCDOWELL-N.NS,914.0,9.59,0.60,1.79,1004.0,448.0,16.19,10.0,L,1.0,1.0,V40N,NaN
3MINDIA.NS,27400.0,17.59,1.13,1.70,29373.0,15332.0,14.05,7.0,M,1.0,1.0,V40N,NaN


In [37]:
# prospects about to turn around
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
#tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] == 1)]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[(tmp_df['Slope%_200'] < 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_200'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 17


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,
ICICIPRULI.NS,574.0,22.18,-0.43,2.53,716.0,238.0,29.71,25.0,M,1.0,0.0,V40,NaN
KANSAINER.NS,455.0,6.13,-0.47,1.92,659.0,292.0,55.59,45.0,M,1.0,0.0,V40N,NaN
TATAELXSI.NS,7599.0,9.76,-1.03,1.78,10602.0,525.0,29.80,40.0,M,0.0,0.0,V40N,NaN
LALPATHLAB.NS,2204.0,1.61,-0.83,1.39,4186.0,807.0,58.66,90.0,M,1.0,0.0,V40N,NaN
HONAUT.NS,41785.0,6.96,-0.07,1.37,48839.0,17924.0,22.82,17.0,M,1.0,0.0,V40N,NaN
GLAXO.NS,1438.0,8.53,-0.08,1.28,1810.0,1026.0,47.45,26.0,M,1.0,0.0,V40,NaN
ERIS.NS,676.0,3.38,-0.15,1.10,828.0,352.0,31.93,22.0,M,1.0,0.0,V40N,NaN
SFL.NS,1210.0,-1.07,-1.39,1.02,1972.0,566.0,54.20,63.0,M,1.0,0.0,V40N,NaN
SIS.NS,410.0,5.40,-0.51,1.01,605.0,321.0,68.66,48.0,M,1.0,0.0,V40N,NaN


In [38]:
# cross-check the features
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
#tmp_df = tmp_df[(tmp_df['BizOutlook'] != 1) | (tmp_df['BizDurability'] != 1)]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[tmp_df['Upside%ATH'] > 40]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 11


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,
JCHAC.NS,1038.0,-11.32,-2.05,-0.65,3219.0,1017.0,99.05,210.0,M,1.0,0.0,V40N,NaN
LALPATHLAB.NS,2204.0,1.61,-0.83,1.39,4186.0,807.0,58.66,90.0,M,1.0,0.0,V40N,NaN
WHIRLPOOL.NS,1464.0,0.84,-1.24,0.93,2667.0,1254.0,85.14,82.0,M,1.0,0.0,V40,NaN
NAM-INDIA.NS,251.0,4.38,-0.83,0.85,425.0,115.0,56.13,69.0,L,1.0,0.0,V40,NaN
SYMPHONY.NS,899.0,-4.56,0.00,-0.99,1470.0,707.0,74.84,64.0,M,1.0,0.0,V40N,NaN
SFL.NS,1210.0,-1.07,-1.39,1.02,1972.0,566.0,54.20,63.0,M,1.0,0.0,V40N,NaN
TTKPRESTIG.NS,725.0,-10.27,-1.36,0.28,1165.0,391.0,56.85,61.0,M,1.0,0.0,V40N,NaN
PFIZER.NS,3845.0,-5.18,-0.47,0.18,6009.0,2283.0,58.08,56.0,L,1.0,0.0,V40,NaN
MOTILALOFS.NS,705.0,5.89,-0.45,0.78,1095.0,441.0,59.63,55.0,L,1.0,0.0,V40N,NaN


In [39]:
# analyse individual stock
stock_id = 'PVRINOX.NS'
df_prec_dev[df_prec_dev.index == stock_id]

,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,
PVRINOX.NS,1381.0,-13.9,-0.87,-1.11,1917.0,1348.0,94.2,39.0,L,0.0,0.0,NaN,NaN


In [40]:
# save the output
df_prec_dev = df_prec_dev.reset_index()
df_prec_dev.to_csv('/content/drive/My Drive/data/stocks/myProspects-Upside-Analysis-ATH.csv', index=False)

In [41]:
# stock in last 5 weeks
#stock_ids = ['ICICIGI','DABUR','MOTILALOFS','JPPOWER','EQUITAS','GLAND','JMFINANCIL','LUXIND','UJJIVANSFB','VAIBHAVGBL','VALIANTORG']
stock_prec_dev('BHARTIARTL.NS')[-21::5]

,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH
Date,,,,,,,,,
2023-05-31 00:00:00+05:30,850.0,BHARTIARTL.NS,8.27,0.76,0.77,872.0,258.0,3.58,3.0
2023-06-07 00:00:00+05:30,844.0,BHARTIARTL.NS,7.14,0.76,1.02,872.0,258.0,4.56,3.0
2023-06-14 00:00:00+05:30,828.0,BHARTIARTL.NS,4.67,0.76,1.01,872.0,258.0,7.17,5.0
2023-06-21 00:00:00+05:30,839.0,BHARTIARTL.NS,5.67,0.76,0.87,872.0,258.0,5.37,4.0
2023-06-28 00:00:00+05:30,872.0,BHARTIARTL.NS,9.55,0.63,1.11,872.0,258.0,0.00,0.0
